# 基于邻域的方法
协同过滤的基本思想是根据用户之前的喜好以及其他兴趣相近的用户的选择来给用户推荐物品（基于对用户历史行为数据的挖掘发现用户的喜好偏向， 并预测用户可能喜好的产品进行推荐），一般是仅仅基于用户的行为数据（评价、购买、下载等），而不依赖于项的任何附加信息（如物品自身特征）或者用户的任何附加信息（年龄、性别等）。目前应用比较广泛的协同过滤算法是基于邻域的方法， 而这种方法主要有下面两种算法：基于用户的协同过滤和基于物品的协同过滤

**缺点**

协同过滤算法存在的问题之一就是泛化能力弱， 即协同过滤无法将两个物品相似的信息推广到其他物品的相似性上。  导致的问题是热门物品具有很强的头部效应， 容易跟大量物品产生相似， 而尾部物品由于特征向量稀疏， 缺乏相似性计算的直接数据，导致很少被推荐。为解决上述问题，同时增加模型的泛化能力，矩阵分解技术（Matrix Factorization，MF）被提出，该方法在协同过滤共现矩阵的基础上，使用更稠密的隐向量表示用户和物品，挖掘用户和物品的隐含兴趣和隐含特征，在一定程度上弥补协同过滤模型处理稀疏矩阵能力不足的问题

##  基于用户的协同过滤（UserCF）

**原理**

该算法假设喜欢相似物品的用户之间，存在着相似的兴趣偏好，从而给用户推荐和他兴趣相似的其他用户喜欢的产品

**步骤**

- 计算用户之间的相似度

    Jaccard 相似度

    $$w_{u,v} = \frac{|\mathcal{N}(u) \cap \mathcal{N}(v)|}{|\mathcal{N}(u) \cup \mathcal{N}(v)|}$$

    其中 $\mathcal{N}(u)$ 表示用户 $u$ 曾经有过正反馈的物品集合

    余弦相似度

    $$w_{u,v} = \frac{|\mathcal{N}(u) \cap \mathcal{N}(v)|}{\sqrt{|\mathcal{N}(u)||\mathcal{N}(v)|}}$$
    
    
    向量相似度。每个用户可以用其评分序列（共享矩阵中的行）表示，这样就可以利用点积等方式计算用户间的相似度

- 找到集合中的用户喜欢的且目标用户没有交互行为的物品推荐给目标用户。这一步的关键是预测目标用户对物品的评价，我们可以利用相似用户的已有评价对目标用户的偏好进行预测。最常用的方式是，利用用户相似度和相似用户的评价加权平均获得用户的评价预测

    $$R_{u, i} = \frac{\sum_{v \in \mathcal{S}(u)} w_{u, v} R_{v, i}}{\sum_{v \in \mathcal{S}(u)} w_{u, v}}$$
    
    其中 $\mathcal{S}(u)$ 表示用户 $u$ 的相似用户集合

    在现实生活中，每个用户的评价标准可能不一样，有的喜欢打高分，有的喜欢打低分，所以我们需要利用用户的平均分对各独立评分进行修正，以减小用户评分偏置的影响
    
    $$R_{u, i} = \bar{R}_u + \frac{\sum_{v \in \mathcal{S}(u)} w_{u, v} \left(R_{v, i} - \bar{R}_v \right)}{\sum_{v \in \mathcal{S}(u)} w_{u, v}}$$
    
**缺点**

- 数据稀疏性。一个大型的电子商务推荐系统一般有非常多的物品，用户可能买的其中不到 1% 的物品，不同用户之间买的物品重叠性较低，导致算法无法找到一个用户的邻居，即偏好相似的用户。这导致 UserCF 不适用于那些正反馈获取较困难的应用场景（如酒店预订，大件商品购买等低频应用）
- 算法可扩展性。基于用户的协同过滤需要维护用户相似度矩阵以便快速的找出 TopN 相似用户， 该矩阵的存储开销非常大，存储空间随着用户数量的增加而增加，不适合用户数据量大的情况使用

**应用场景**

适合用户少，物品多，时效性较强的场合，如新闻推荐

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 定义数据集， 也就是那个表格， 注意这里我们采用字典存放数据， 因为实际情况中数据是非常稀疏的
def loadData():
    items = {'A': {1: 5, 2: 3, 3: 4, 4: 3, 5: 1},
             'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5},
             'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5},
             'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2},
             'E': {2: 3, 3: 5, 4: 4, 5: 1}
             }
    users = {1: {'A': 5, 'B': 3, 'C': 4, 'D': 4},
             2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
             3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
             4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
             5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
             }
    return items, users


items, users = loadData()
item_df = pd.DataFrame(items).T
user_df = pd.DataFrame(users).T

In [3]:
item_df

,1,2,3,4,5
A,5.0,3.0,4.0,3.0,1.0
B,3.0,1.0,3.0,3.0,5.0
C,4.0,2.0,4.0,1.0,5.0
D,4.0,3.0,3.0,5.0,2.0
E,NaN,3.0,5.0,4.0,1.0


In [4]:
user_df

,A,B,C,D,E
1,5.0,3.0,4.0,4.0,NaN
2,3.0,1.0,2.0,3.0,3.0
3,4.0,3.0,4.0,3.0,5.0
4,3.0,3.0,1.0,5.0,4.0
5,1.0,5.0,5.0,2.0,1.0


In [5]:
"""计算用户相似性矩阵"""
similarity_matrix = pd.DataFrame(np.zeros((len(users), len(users))),
                                 index=[1, 2, 3, 4, 5],
                                 columns=[1, 2, 3, 4, 5])

# 遍历每条用户-物品评分数据
for userId in users:
    for otheruserId in users:
        vec_user = []
        vec_otheruser = []
        if userId != otheruserId:
            for itemId in items:  # 遍历物品-用户评分数据
                itemRatings = items[itemId]  # 每条数据为所有用户对当前物品的评分
                if userId in itemRatings and otheruserId in itemRatings:  # 如果两个用户都对该物品评过分
                    vec_user.append(itemRatings[userId])
                    vec_otheruser.append(itemRatings[otheruserId])
            # 这里可以获得相似性矩阵（共现矩阵），采用皮尔逊相关系数
            similarity_matrix[userId][otheruserId] = np.corrcoef(
                np.array(vec_user), np.array(vec_otheruser))[0][1]

In [6]:
"""计算前n个相似的用户"""
n = 2
similarity_users = similarity_matrix[1].sort_values(ascending=False)[:n].index.tolist() # [2, 3] 也就是用户1和用户2

In [7]:
similarity_matrix

,1,2,3,4,5
1,0.000000,0.852803,0.707107,0.000000,-0.792118
2,0.852803,0.000000,0.467707,0.489956,-0.900149
3,0.707107,0.467707,0.000000,-0.161165,-0.466569
4,0.000000,0.489956,-0.161165,0.000000,-0.641503
5,-0.792118,-0.900149,-0.466569,-0.641503,0.000000


In [8]:
"""计算最终得分"""
base_score = np.mean(np.array([value for value in users[1].values()]))
weighted_scores = 0.
corr_values_sum = 0.
for user in similarity_users:  # [2, 3]
    corr_value = similarity_matrix[1][user]  # 两个用户之间的相似性
    mean_user_score = np.mean(np.array([value for value in users[user].values()]))  # 每个用户的打分平均值
    weighted_scores += corr_value * (users[user]['E'] - mean_user_score)  # 加权分数
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores / corr_values_sum
print('用户Alice对物品5的打分: ', final_scores)
user_df.loc[1]['E'] = final_scores
user_df

用户Alice对物品5的打分:  4.871979899370592


,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.87198
2,3.0,1.0,2.0,3.0,3.00000
3,4.0,3.0,4.0,3.0,5.00000
4,3.0,3.0,1.0,5.0,4.00000
5,1.0,5.0,5.0,2.0,1.00000


## 基于物品的协同过滤（ItemCF）

**原理**

该算法假设用户对于不同物品间的偏好相似，那么物品间也是相似的，从而给用户推荐和他之前喜欢的物品相似的物品。例如该算法会因为你购买过《数据挖掘导论》而给你推荐《机器学习》。不过，ItemCF 算法并不利用物品的内容属性计算物品之间的相似度，它主要通过分析用户的行为记录计算物品之间的相似度

**步骤**

- 计算物品之间的相似度

    从定义出发，可以得到物品相似度计算公式

    $$w_{i,j} = \frac{|\mathcal{N}(i) \cap \mathcal{N}(j)|}{|\mathcal{N}(i)|}$$
    
    其中 $\mathcal{N}(i)$ 喜欢物品 $i$ 的用户集合

    为了避免推荐出热门的物品，对公式进行改进

    $$w_{i,j} = \frac{|\mathcal{N}(i) \cap \mathcal{N}(j)|}{\sqrt{|\mathcal{N}(i)||\mathcal{N}(j)|}}$$

    同样，每个物品可以用其用户评分序列表示（共现矩阵中的列），然后再利用点积等方式计算物品间的相似度


- 根据物品的相似度和用户的历史行为给用户生成推荐列表

    $$R_{u, i} = \bar{R}_i + \frac{\sum_{j \in \mathcal{S}(i)} w_{i, j} \left(R_{u, j} - \bar{R}_j \right)}{\sum_{j \in \mathcal{S}(i)} w_{i, j}}$$

    其中 $\mathcal{S}(i)$ 表示物品 $i$ 的相似物品集合  

**应用场景**

物品少，用户多，用户兴趣较为稳定，物品更新速度不是太快的场合，如图书、电子商务和电影网站

In [9]:
"""计算物品的相似矩阵"""
similarity_matrix = pd.DataFrame(np.zeros((len(items), len(items))),
                                 index=['A', 'B', 'C', 'D', 'E'],
                                 columns=['A', 'B', 'C', 'D', 'E'])

# 遍历每条物品-用户评分数据
for itemId in items:
    for otheritemId in items:
        vec_item = []  # 定义列表， 保存当前两个物品的向量值
        vec_otheritem = []
        if itemId != otheritemId:  # 物品不同
            for userId in users:  # 遍历用户-物品评分数据
                userRatings = users[userId]  # 每条数据为该用户对所有物品的评分

                if itemId in userRatings and otheritemId in userRatings:  # 用户对这两个物品都评过分
                    vec_item.append(userRatings[itemId])
                    vec_otheritem.append(userRatings[otheritemId])

            # 这里可以获得相似性矩阵（共现矩阵），采用皮尔逊相关系数
            similarity_matrix[itemId][otheritemId] = np.corrcoef(
                np.array(vec_item), np.array(vec_otheritem))[0][1]

In [10]:
"""得到与物品5相似的前n个物品"""
n = 2
similarity_items = similarity_matrix['E'].sort_values(ascending=False)[:n].index.tolist()  # ['A', 'D']

In [11]:
similarity_matrix

,A,B,C,D,E
A,0.000000,-0.476731,-0.123091,0.532181,0.969458
B,-0.476731,0.000000,0.645497,-0.310087,-0.478091
C,-0.123091,0.645497,0.000000,-0.720577,-0.427618
D,0.532181,-0.310087,-0.720577,0.000000,0.581675
E,0.969458,-0.478091,-0.427618,0.581675,0.000000


In [12]:
"""计算最终得分"""
base_score = np.mean(np.array([value for value in items['E'].values()]))
weighted_scores = 0.
corr_values_sum = 0.
for item in similarity_items:  # ['A', 'D']
    corr_value = similarity_matrix['E'][item]  # 两个物品之间的相似性
    mean_item_score = np.mean(np.array([value for value in items[item].values()]))  # 每个物品的打分平均值
    weighted_scores += corr_value * (users[1][item] - mean_item_score)  # 加权分数
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores / corr_values_sum
print('用户Alice对物品5的打分: ', final_scores)
user_df.loc[1]['E'] = final_scores
user_df

用户Alice对物品5的打分:  4.6


,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.6
2,3.0,1.0,2.0,3.0,3.0
3,4.0,3.0,4.0,3.0,5.0
4,3.0,3.0,1.0,5.0,4.0
5,1.0,5.0,5.0,2.0,1.0
